<a href="https://colab.research.google.com/github/MarcoParola/structural_health_monitoring/blob/main/testing_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Testing Convolutional models**

In this notebook we show an evaluation of the models built and trained on "convolutional_models.ipynb". More in detail we want to observe how the models trained on some conditions (signal corruption) work in different scenarios

## Link Drive, import packages and declare global variables

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import r2_score, mean_absolute_error, confusion_matrix
import tensorflow as tf

projectPath = 'drive/MyDrive/Colab Notebooks/thesis2/'

NUMBER_OF_CLASS = 9
TRUNCATE = 9000

# the most usefull sensors are: 1,5,9,12,14,15
filter_indeces = [1,5,9,12,14,15]

earlyStopCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                     mode='min', verbose=1, 
                                                     patience=20, 
                                                     restore_best_weights=True)

In [3]:
%cd $projectPath
%run utils.ipynb
%cd '../../../..'

/content/drive/.shortcut-targets-by-id/1RdMs44loEcFGBtBOenib5G7znvrr04KT/thesis2
/content


Training set and test are already divided

In [4]:
data = np.load(DATASET)

labels = load_dataset('damage_labels')
levels = load_dataset('damage_levels')

randomize = np.arange(len(labels))
np.random.shuffle(randomize)
data = data[randomize,:,:]
labels = labels[randomize]
levels = levels[randomize]

dataset = data[0:TRUNCATE, :, :]
testset = data[TRUNCATE:len(data), :, :]

labels_train = labels[0:TRUNCATE]
levels_train = levels[0:TRUNCATE]
labels_test = labels[TRUNCATE:len(labels)]
levels_test = levels[TRUNCATE:len(levels)]

labels_train = to_categorical(labels_train)
labels_test = to_categorical(labels_test)

print(dataset.shape)
print(testset.shape)

(9999,)
(9999,)
(9000, 1751, 6)
(999, 1751, 6)


# **Classification**

### Testing classifier trained using clean data on noisy and desynchronized data

In [ ]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_classifier.h5"))

for delay in range(1,12):
  for noise_i in range(12):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateClassifier(model, t, levels_test)

### Testing classifier trained using noisy data on noisy and desynchronized data

In [ ]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_noisy_classifier.h5"))

for delay in range(1,12):
  for noise_i in range(12):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateClassifier(model, t, levels_test)

--------------------- 1 0 ---------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.58      0.92      0.71       131
           1       0.67      0.26      0.38        99
           2       0.91      1.00      0.95       124
           3       0.53      0.74      0.62       117
           4       1.00      0.86      0.93       111
           5       1.00      0.97      0.99       113
           6       1.00      0.53      0.70       120
           7       1.00      0.99      0.99       100
           8       1.00      1.00      1.00        84

    accuracy                           0.81       999
   macro avg       0.85      0.81      0.81       999
weighted avg       0.85      0.81      0.80       999

--------------------- 1 1 ---------------------
SNR: 54.64819607401516 

              precision    recall  f1-score   support

           0       0.71      0.92      0.80       131
           1       0.67      0.52      0.58        99
           2       0.89      1.00      0.94     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.41      0.53      0.46       131
           1       0.36      0.15      0.21        99
           2       0.62      0.90      0.74       124
           3       0.44      0.44      0.44       117
           4       1.00      0.73      0.84       111
           5       0.93      0.95      0.94       113
           6       0.83      0.53      0.65       120
           7       0.82      0.85      0.83       100
           8       0.68      0.93      0.78        84

    accuracy                           0.66       999
   macro avg       0.68      0.67      0.66       999
weighted avg       0.67      0.66      0.65       999

--------------------- 2 1 ---------------------
SNR: 54.652768105912024 

              precision    recall  f1-score   support

           0       0.43      0.48      0.46       131
           1       0.28      0.19      0.23        99
           2       0.60      0.89      0.72    

...

SNR: inf 

              precision    recall  f1-score   support

           0       0.06      0.02      0.03       131
           1       0.29      0.02      0.04        99
           2       0.16      0.20      0.18       124
           3       0.12      0.04      0.06       117
           4       0.16      0.17      0.16       111
           5       0.20      0.28      0.23       113
           6       0.18      0.24      0.21       120
           7       0.12      0.13      0.13       100
           8       0.12      0.30      0.18        84

    accuracy                           0.15       999
   macro avg       0.16      0.16      0.14       999
weighted avg       0.16      0.15      0.13       999

--------------------- 9 1 ---------------------
SNR: 54.64593771950557 

              precision    recall  f1-score   support

           0       0.14      0.05      0.07       131
           1       0.11      0.02      0.03        99
           2       0.15      0.22      0.18     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.13      0.05      0.08       131
           1       0.08      0.01      0.02        99
           2       0.11      0.13      0.12       124
           3       0.17      0.08      0.11       117
           4       0.18      0.19      0.18       111
           5       0.23      0.34      0.27       113
           6       0.15      0.23      0.18       120
           7       0.18      0.18      0.18       100
           8       0.12      0.24      0.16        84

    accuracy                           0.16       999
   macro avg       0.15      0.16      0.14       999
weighted avg       0.15      0.16      0.14       999

--------------------- 10 1 ---------------------
SNR: 54.59588452483428 

              precision    recall  f1-score   support

           0       0.20      0.05      0.07       131
           1       0.06      0.01      0.02        99
           2       0.13      0.19      0.15    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.09      0.03      0.05       131
           1       0.14      0.02      0.04        99
           2       0.15      0.19      0.17       124
           3       0.04      0.02      0.02       117
           4       0.10      0.09      0.10       111
           5       0.18      0.27      0.22       113
           6       0.12      0.18      0.14       120
           7       0.16      0.15      0.15       100
           8       0.11      0.23      0.15        84

    accuracy                           0.13       999
   macro avg       0.12      0.13      0.11       999
weighted avg       0.12      0.13      0.11       999

--------------------- 11 1 ---------------------
SNR: 54.63101891929241 

              precision    recall  f1-score   support

           0       0.13      0.04      0.06       131
           1       0.00      0.00      0.00        99
           2       0.11      0.14      0.12    

### Testing classifier trained using desynchronized data on noisy and desynchronized data

In [11]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_desynch_classifier.h5"))

for delay in range(1,10):
  for noise_i in range(10):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateClassifier(model, t, levels_test)

--------------------- 1 0 ---------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.79      0.94      0.86       113
           1       0.92      0.67      0.77       115
           2       1.00      0.95      0.97       110
           3       0.67      0.53      0.59       128
           4       1.00      0.92      0.96       112
           5       1.00      1.00      1.00       104
           6       0.64      0.99      0.77        99
           7       1.00      0.96      0.98        98
           8       1.00      1.00      1.00       120

    accuracy                           0.87       999
   macro avg       0.89      0.88      0.88       999
weighted avg       0.89      0.87      0.87       999

--------------------- 1 1 ---------------------
SNR: 54.79894574455782 

              precision    recall  f1-score   support

           0       0.51      0.35      0.41       113
           1       0.17      0.03      0.06       115
           2       1.00      0.51      0.67     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       113
           1       0.00      0.00      0.00       115
           2       0.00      0.00      0.00       110
           3       0.00      0.00      0.00       128
           4       0.53      0.15      0.24       112
           5       0.54      0.76      0.63       104
           6       0.26      0.83      0.39        99
           7       1.00      0.31      0.47        98
           8       0.27      1.00      0.42       120

    accuracy                           0.33       999
   macro avg       0.29      0.34      0.24       999
weighted avg       0.27      0.33      0.23       999

--------------------- 1 4 ---------------------
SNR: 3.4215900188097046 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       113
           1       0.00      0.00      0.00       115
           2       1.00      0.01      0.02       110
    

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       113
           1       0.00      0.00      0.00       115
           2       0.00      0.00      0.00       110
           3       0.00      0.00      0.00       128
           4       0.00      0.00      0.00       112
           5       0.84      0.25      0.39       104
           6       0.10      0.16      0.12        99
           7       1.00      0.02      0.04        98
           8       0.15      1.00      0.26       120

    accuracy                           0.16       999
   macro avg       0.23      0.16      0.09       999
weighted avg       0.21      0.16      0.09       999

--------------------- 9 8 ---------------------
SNR: 0.8558845789972872 



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       113
           1       0.00      0.00      0.00       115
           2       0.00      0.00      0.00       110
           3       0.00      0.00      0.00       128
           4       0.00      0.00      0.00       112
           5       0.85      0.16      0.27       104
           6       0.13      0.19      0.15        99
           7       0.00      0.00      0.00        98
           8       0.15      1.00      0.25       120

    accuracy                           0.16       999
   macro avg       0.12      0.15      0.08       999
weighted avg       0.12      0.16      0.07       999

--------------------- 9 9 ---------------------
SNR: 0.6767363147929042 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       113
           1       0.00      0.00      0.00       115
           2       0.00      0.00      0.00       110
    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Testing classifier trained using noisy and desynchronized data on noisy and desynchronized data

In [12]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_noisy_desync_classifier.h5"))

for delay in range(1,10):
  for noise_i in range(10):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateClassifier(model, t, levels_test)

--------------------- 1 0 ---------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.66      0.88      0.76       113
           1       0.54      0.17      0.26       115
           2       0.98      0.98      0.98       110
           3       0.46      0.66      0.55       128
           4       0.98      0.84      0.90       112
           5       1.00      0.98      0.99       104
           6       0.70      0.82      0.76        99
           7       1.00      0.88      0.93        98
           8       1.00      0.99      1.00       120

    accuracy                           0.80       999
   macro avg       0.81      0.80      0.79       999
weighted avg       0.81      0.80      0.78       999

--------------------- 1 1 ---------------------
SNR: 54.76124014102938 

              precision    recall  f1-score   support

           0       0.60      0.81      0.69       113
           1       0.58      0.29      0.38       115
           2       0.94      0.94      0.94     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.63      0.84      0.72       113
           1       0.54      0.19      0.28       115
           2       0.96      0.97      0.97       110
           3       0.49      0.66      0.56       128
           4       0.98      0.88      0.93       112
           5       1.00      0.97      0.99       104
           6       0.69      0.80      0.74        99
           7       1.00      0.86      0.92        98
           8       1.00      1.00      1.00       120

    accuracy                           0.79       999
   macro avg       0.81      0.80      0.79       999
weighted avg       0.80      0.79      0.78       999

--------------------- 2 1 ---------------------
SNR: 54.7830954689231 

              precision    recall  f1-score   support

           0       0.63      0.81      0.71       113
           1       0.54      0.27      0.36       115
           2       0.96      0.95      0.95      

...

SNR: inf 

              precision    recall  f1-score   support

           0       0.50      0.58      0.54       113
           1       0.48      0.26      0.34       115
           2       0.89      0.87      0.88       110
           3       0.44      0.66      0.53       128
           4       0.92      0.86      0.89       112
           5       0.99      0.91      0.95       104
           6       0.77      0.74      0.75        99
           7       0.94      0.87      0.90        98
           8       0.97      0.97      0.97       120

    accuracy                           0.74       999
   macro avg       0.77      0.75      0.75       999
weighted avg       0.76      0.74      0.74       999

--------------------- 7 1 ---------------------
SNR: 54.74075021406535 

              precision    recall  f1-score   support

           0       0.58      0.63      0.60       113
           1       0.49      0.41      0.45       115
           2       0.90      0.91      0.90     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.54      0.61      0.57       113
           1       0.38      0.19      0.25       115
           2       0.86      0.90      0.88       110
           3       0.43      0.62      0.51       128
           4       0.86      0.81      0.83       112
           5       1.00      0.88      0.93       104
           6       0.68      0.74      0.71        99
           7       0.90      0.83      0.86        98
           8       0.97      0.94      0.95       120

    accuracy                           0.72       999
   macro avg       0.73      0.72      0.72       999
weighted avg       0.73      0.72      0.72       999

--------------------- 8 1 ---------------------
SNR: 54.759522527043 

              precision    recall  f1-score   support

           0       0.52      0.49      0.50       113
           1       0.41      0.36      0.38       115
           2       0.82      0.85      0.83       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.52      0.54      0.53       113
           1       0.38      0.22      0.28       115
           2       0.88      0.90      0.89       110
           3       0.41      0.58      0.48       128
           4       0.85      0.82      0.84       112
           5       0.99      0.87      0.92       104
           6       0.73      0.77      0.75        99
           7       0.84      0.83      0.84        98
           8       0.92      0.93      0.93       120

    accuracy                           0.71       999
   macro avg       0.72      0.72      0.72       999
weighted avg       0.72      0.71      0.71       999

--------------------- 9 1 ---------------------
SNR: 54.75621052012504 

              precision    recall  f1-score   support

           0       0.50      0.44      0.47       113
           1       0.31      0.27      0.29       115
           2       0.82      0.83      0.82     

# **Regression**

### Testing regressor trained using clean data on noisy and desynchronized data

In [ ]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_regressor.h5"))

for delay in range(1,12):
  for noise_i in range(12):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateRegressor(model, t, levels_test)

--------------------- 1 0 ---------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  0.8147476446068015 
MAE 0.01557650449142394
--------------------- 1 1 ---------------------
SNR: 56.63638525647478 

R^2:  -38.927296256835994 
MAE 0.3526025311089822
--------------------- 1 2 ---------------------
SNR: 14.160435475395113 

R^2:  -38.06531605049484 
MAE 0.34812177992055315
--------------------- 1 3 ---------------------
SNR: 6.2888328760237915 

R^2:  -35.38085617601947 
MAE 0.33554540998081017
--------------------- 1 4 ---------------------
SNR: 3.536759500705271 

R^2:  -47.22211434254429 
MAE 0.3872440171692555
--------------------- 1 5 ---------------------
SNR: 2.2635830010329587 

R^2:  -64.18174453352616 
MAE 0.451302031958518
--------------------- 1 6 ---------------------
SNR: 1.5730269838700401 

R^2:  -84.86518704007798 
MAE 0.5188602735769254
--------------------- 1 7 ---------------------
SNR: 1.1548699566221918 

R^2:  -109.75749945932125 
MAE 0.5899501118177618
--------------------- 1 8 ---------------------
SNR: 0.8843224612087613 

R^2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -11.621931590318576 
MAE 0.17381319361247147
--------------------- 2 1 ---------------------
SNR: 56.59079693971379 

R^2:  -66.91241776616745 
MAE 0.45613271509536346
--------------------- 2 2 ---------------------
SNR: 14.157325611228247 

R^2:  -59.01345272046906 
MAE 0.4286851078203581
--------------------- 2 3 ---------------------
SNR: 6.287367744781969 

R^2:  -49.964796119532934 
MAE 0.39624989114697584
--------------------- 2 4 ---------------------
SNR: 3.5360937986479737 

R^2:  -59.628314994324 
MAE 0.43342134784503933
--------------------- 2 5 ---------------------
SNR: 2.2637228892113304 

R^2:  -76.54404922224418 
MAE 0.4916136297458614
--------------------- 2 6 ---------------------
SNR: 1.5728364553007994 

R^2:  -97.06260835332806 
MAE 0.5540046901158475
--------------------- 2 7 ---------------------
SNR: 1.1551475059515743 

R^2:  -121.05291854765579 
MAE 0.6189465696828115
--------------------- 2 8 ---------------------
SNR: 0.884553340700022 

R^2

...

SNR: inf 

R^2:  -140.3207315598405 
MAE 0.5625017000496609
--------------------- 9 1 ---------------------
SNR: 56.60935690042858 

R^2:  -237.5251903741203 
MAE 0.8048264350103277
--------------------- 9 2 ---------------------
SNR: 14.155188401641036 

R^2:  -215.69331144527484 
MAE 0.7686635447178249
--------------------- 9 3 ---------------------
SNR: 6.292429770400486 

R^2:  -184.25880967539848 
MAE 0.7134838832889802
--------------------- 9 4 ---------------------
SNR: 3.5398872209877728 

R^2:  -196.26071798303988 
MAE 0.7428554313041755
--------------------- 9 5 ---------------------
SNR: 2.264428179469281 

R^2:  -209.4395308818996 
MAE 0.7779889331527152
--------------------- 9 6 ---------------------
SNR: 1.573309058982125 

R^2:  -231.17149638548247 
MAE 0.8239926394957771
--------------------- 9 7 ---------------------
SNR: 1.1554378438188808 

R^2:  -263.0297113457597 
MAE 0.8849068060135789
--------------------- 9 8 ---------------------
SNR: 0.8842407161919733 

R^2: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -150.70102654486956 
MAE 0.5964692314487052
--------------------- 10 1 ---------------------
SNR: 56.60487475726584 

R^2:  -262.8654928097169 
MAE 0.8363948177250508
--------------------- 10 2 ---------------------
SNR: 14.146488885923995 

R^2:  -229.3257259530927 
MAE 0.7886760713481062
--------------------- 10 3 ---------------------
SNR: 6.288120300278675 

R^2:  -208.04892535135025 
MAE 0.7540703819938579
--------------------- 10 4 ---------------------
SNR: 3.5413842376318643 

R^2:  -220.03001723992114 
MAE 0.7775967092924261
--------------------- 10 5 ---------------------
SNR: 2.266394450822338 

R^2:  -231.2913446395832 
MAE 0.8101642307657351
--------------------- 10 6 ---------------------
SNR: 1.573700138253824 

R^2:  -255.0441881318863 
MAE 0.862269857131179
--------------------- 10 7 ---------------------
SNR: 1.1549858737420564 

R^2:  -275.90917449233314 
MAE 0.9052353063822719
--------------------- 10 8 ---------------------
SNR: 0.8857867232779995 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -157.6942655594029 
MAE 0.6103143080299652
--------------------- 11 1 ---------------------
SNR: 56.61521275917697 

R^2:  -280.47898103407726 
MAE 0.8670426413661816
--------------------- 11 2 ---------------------
SNR: 14.160573201538744 

R^2:  -253.93782222193298 
MAE 0.8217883921323209
--------------------- 11 3 ---------------------
SNR: 6.292406422901081 

R^2:  -225.83434409877927 
MAE 0.7804290470141162
--------------------- 11 4 ---------------------
SNR: 3.5412311147542743 

R^2:  -234.91367501077613 
MAE 0.8042255299785565
--------------------- 11 5 ---------------------
SNR: 2.2639918983041225 

R^2:  -240.98856339759016 
MAE 0.8279339595800138
--------------------- 11 6 ---------------------
SNR: 1.572006871172918 

R^2:  -275.97240107536055 
MAE 0.8915320517742092
--------------------- 11 7 ---------------------
SNR: 1.1555514067568726 

R^2:  -294.2176598724226 
MAE 0.9335557305141942
--------------------- 11 8 ---------------------
SNR: 0.8846336931990

### Testing regressor trained using noisy data on noisy and desynchronized data

In [ ]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_noisy_regressor.h5"))

for delay in range(1,12):
  for noise_i in range(12):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateRegressor(model, t, levels_test)

--------------------- 1 0 ---------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -3.207517304791552 
MAE 0.10500809055840277
--------------------- 1 1 ---------------------
SNR: 56.6364851496526 

R^2:  -2.3438323629625604 
MAE 0.09142722536513923
--------------------- 1 2 ---------------------
SNR: 14.157643397111691 

R^2:  -1.1815246905758592 
MAE 0.06917051426487283
--------------------- 1 3 ---------------------
SNR: 6.291881369338687 

R^2:  -0.41887727655736806 
MAE 0.0522319883761184
--------------------- 1 4 ---------------------
SNR: 3.5362048849266 

R^2:  0.062309318009946324 
MAE 0.042269602245797615
--------------------- 1 5 ---------------------
SNR: 2.265257307893074 

R^2:  0.20248935003694823 
MAE 0.04094957700808175
--------------------- 1 6 ---------------------
SNR: 1.5735790679255433 

R^2:  0.12637308774308753 
MAE 0.04391632161939418
--------------------- 1 7 ---------------------
SNR: 1.155642648562956 

R^2:  -0.2606657511087689 
MAE 0.052644831814543505
--------------------- 1 8 ---------------------
SNR: 0.88428075204014

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -2.712150478474621 
MAE 0.09648854988612118
--------------------- 2 1 ---------------------
SNR: 56.65732619251253 

R^2:  -2.0221202147291333 
MAE 0.08475344817166872
--------------------- 2 2 ---------------------
SNR: 14.160507510530236 

R^2:  -1.0253968808721186 
MAE 0.0648879847083019
--------------------- 2 3 ---------------------
SNR: 6.293880104988117 

R^2:  -0.32056471387712016 
MAE 0.051281881339745955
--------------------- 2 4 ---------------------
SNR: 3.538498213088436 

R^2:  0.05166807883718516 
MAE 0.04392574417781114
--------------------- 2 5 ---------------------
SNR: 2.264830337621488 

R^2:  0.14493104020403258 
MAE 0.042751774124010426
--------------------- 2 6 ---------------------
SNR: 1.5730077658845951 

R^2:  -0.0022745942611104297 
MAE 0.04737059478538244
--------------------- 2 7 ---------------------
SNR: 1.1562615263569642 

R^2:  -0.4360750529068764 
MAE 0.05625639762788397
--------------------- 2 8 ---------------------
SNR: 0.88493867

...

SNR: inf 

R^2:  -7.124693199994429 
MAE 0.1142008529215636
--------------------- 9 1 ---------------------
SNR: 56.62599162922192 

R^2:  -5.769519893049856 
MAE 0.10871926652098395
--------------------- 9 2 ---------------------
SNR: 14.154360968254055 

R^2:  -6.662783091535009 
MAE 0.1128098200171365
--------------------- 9 3 ---------------------
SNR: 6.291198684936014 

R^2:  -6.506455831897206 
MAE 0.11155066094648804
--------------------- 9 4 ---------------------
SNR: 3.5357462783262656 

R^2:  -6.306348446339186 
MAE 0.11212955059908818
--------------------- 9 5 ---------------------
SNR: 2.263719229646384 

R^2:  -6.784536840163713 
MAE 0.11888489361064165
--------------------- 9 6 ---------------------
SNR: 1.573324486977791 

R^2:  -8.266932086638397 
MAE 0.13143589880869028
--------------------- 9 7 ---------------------
SNR: 1.1547773048939272 

R^2:  -8.0306187554485 
MAE 0.1323341924400893
--------------------- 9 8 ---------------------
SNR: 0.885107464258668 

R^2:  -

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -7.444881070596935 
MAE 0.12058015636938292
--------------------- 10 1 ---------------------
SNR: 56.63177893972687 

R^2:  -7.224901569395774 
MAE 0.11597674878932132
--------------------- 10 2 ---------------------
SNR: 14.16085538733133 

R^2:  -8.175453510190247 
MAE 0.12117751629769469
--------------------- 10 3 ---------------------
SNR: 6.295365247840707 

R^2:  -7.463735606146633 
MAE 0.11892253592480412
--------------------- 10 4 ---------------------
SNR: 3.5396406039177335 

R^2:  -8.138900394331207 
MAE 0.12502837939577133
--------------------- 10 5 ---------------------
SNR: 2.2639821396218855 

R^2:  -8.218591830599726 
MAE 0.12871412635367865
--------------------- 10 6 ---------------------
SNR: 1.5717179911662553 

R^2:  -8.401112310284065 
MAE 0.13303791214611388
--------------------- 10 7 ---------------------
SNR: 1.1557989213003192 

R^2:  -10.79142106120488 
MAE 0.15150618771042504
--------------------- 10 8 ---------------------
SNR: 0.88476270590

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -9.377642283348184 
MAE 0.12733930943849817
--------------------- 11 1 ---------------------
SNR: 56.59479690937643 

R^2:  -7.871062066874382 
MAE 0.12084024396977221
--------------------- 11 2 ---------------------
SNR: 14.155335834921026 

R^2:  -7.979408317301832 
MAE 0.12410177984378669
--------------------- 11 3 ---------------------
SNR: 6.294444094247714 

R^2:  -8.89349649542241 
MAE 0.12926683655800186
--------------------- 11 4 ---------------------
SNR: 3.5391269098016376 

R^2:  -9.014594325437107 
MAE 0.13233899603188026
--------------------- 11 5 ---------------------
SNR: 2.2661038528587047 

R^2:  -9.71803099134202 
MAE 0.13992305370725025
--------------------- 11 6 ---------------------
SNR: 1.573122412005497 

R^2:  -10.035053723951814 
MAE 0.14733196466684292
--------------------- 11 7 ---------------------
SNR: 1.1553113738652072 

R^2:  -11.424124557246719 
MAE 0.15749966306667498
--------------------- 11 8 ---------------------
SNR: 0.88515971954

### Testing regressor trained using desynchronized data on noisy and desynchronized data

In [5]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_desynch_regressor.h5"))

for delay in range(1,10):
  for noise_i in range(10):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateRegressor(model, t, levels_test)

--------------------- 1 0 ---------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  0.35367301418393526 
MAE 0.038772619967114584
--------------------- 1 1 ---------------------
SNR: 54.723278050887636 

R^2:  -0.6714176063931432 
MAE 0.06431318366073918
--------------------- 1 2 ---------------------
SNR: 13.67577969468783 

R^2:  -7.10347739054612 
MAE 0.15705376391924228
--------------------- 1 3 ---------------------
SNR: 6.08062121709979 

R^2:  -19.67036778132467 
MAE 0.2598955326052428
--------------------- 1 4 ---------------------
SNR: 3.4194370513555388 

R^2:  -39.04261457658201 
MAE 0.3655984440762269
--------------------- 1 5 ---------------------
SNR: 2.1874276544301954 

R^2:  -64.80161362542987 
MAE 0.47190713870306084
--------------------- 1 6 ---------------------
SNR: 1.519729003560521 

R^2:  -96.84965211856924 
MAE 0.5772810731066149
--------------------- 1 7 ---------------------
SNR: 1.1155926223678372 

R^2:  -135.8821579905811 
MAE 0.6846332799424906
--------------------- 1 8 ---------------------
SNR: 0.8550318175895901 

R^2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  0.35129827869379004 
MAE 0.03866780229651583
--------------------- 2 1 ---------------------
SNR: 54.720988074691945 

R^2:  -0.6919876234887792 
MAE 0.06477191410620815
--------------------- 2 2 ---------------------
SNR: 13.674993432026039 

R^2:  -7.255273211211277 
MAE 0.15834109677359434
--------------------- 2 3 ---------------------
SNR: 6.0807230706826365 

R^2:  -19.8656362321606 
MAE 0.26099773498419454
--------------------- 2 4 ---------------------
SNR: 3.4194703698674562 

R^2:  -38.805835162998626 
MAE 0.36468460388997703
--------------------- 2 5 ---------------------
SNR: 2.18716338873814 

R^2:  -64.15798474984568 
MAE 0.46954634203717804
--------------------- 2 6 ---------------------
SNR: 1.5192140466771211 

R^2:  -96.56504175116068 
MAE 0.576743453888161
--------------------- 2 7 ---------------------
SNR: 1.1167027376018392 

R^2:  -134.71246141696128 
MAE 0.6817191808171964
--------------------- 2 8 ---------------------
SNR: 0.8540887759047389 


...

SNR: inf 

R^2:  0.16381093292500948 
MAE 0.044591254791333274
--------------------- 7 1 ---------------------
SNR: 54.70443903802033 

R^2:  -0.8434505972534796 
MAE 0.06679586680528804
--------------------- 7 2 ---------------------
SNR: 13.667765443383514 

R^2:  -6.798862304856065 
MAE 0.1532396863775836
--------------------- 7 3 ---------------------
SNR: 6.078874570606603 

R^2:  -18.702472415432094 
MAE 0.2529498996477614
--------------------- 7 4 ---------------------
SNR: 3.4213109245063573 

R^2:  -37.05450931251374 
MAE 0.3564603582257413
--------------------- 7 5 ---------------------
SNR: 2.187511938000147 

R^2:  -61.552999606674824 
MAE 0.46000966286308703
--------------------- 7 6 ---------------------
SNR: 1.5197629552454586 

R^2:  -92.00023036300693 
MAE 0.5626250939782823
--------------------- 7 7 ---------------------
SNR: 1.1163712662045877 

R^2:  -131.36903918606998 
MAE 0.673340233952508
--------------------- 7 8 ---------------------
SNR: 0.8549201090915677 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -0.10970133282579408 
MAE 0.04992716081642174
--------------------- 8 1 ---------------------
SNR: 54.70607887018201 

R^2:  -0.9512334880034796 
MAE 0.06817219318310228
--------------------- 8 2 ---------------------
SNR: 13.670164880998502 

R^2:  -6.8776657683953895 
MAE 0.15260050906197375
--------------------- 8 3 ---------------------
SNR: 6.078429888084122 

R^2:  -18.829709442714847 
MAE 0.2531195534698746
--------------------- 8 4 ---------------------
SNR: 3.4177613770511783 

R^2:  -36.93543064305554 
MAE 0.35528006072548224
--------------------- 8 5 ---------------------
SNR: 2.1889528527303903 

R^2:  -60.35498177122604 
MAE 0.45496198394547044
--------------------- 8 6 ---------------------
SNR: 1.5207986283309147 

R^2:  -91.69927092473881 
MAE 0.5617733524049275
--------------------- 8 7 ---------------------
SNR: 1.117256140066882 

R^2:  -127.72710747565986 
MAE 0.6633627696894817
--------------------- 8 8 ---------------------
SNR: 0.855450486602593 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -0.25378159473533746 
MAE 0.0533169505271272
--------------------- 9 1 ---------------------
SNR: 54.719030240659386 

R^2:  -1.2724683307079445 
MAE 0.07234705680597342
--------------------- 9 2 ---------------------
SNR: 13.677854391325567 

R^2:  -7.355747414304073 
MAE 0.15698153373998733
--------------------- 9 3 ---------------------
SNR: 6.077219042728507 

R^2:  -19.280276452848216 
MAE 0.2560395539644106
--------------------- 9 4 ---------------------
SNR: 3.4214170704428013 

R^2:  -36.643393054143054 
MAE 0.3528929958033996
--------------------- 9 5 ---------------------
SNR: 2.189342111375769 

R^2:  -59.99471596924139 
MAE 0.4527399381816356
--------------------- 9 6 ---------------------
SNR: 1.5196510081536525 

R^2:  -90.16725614470332 
MAE 0.5563639965003226
--------------------- 9 7 ---------------------
SNR: 1.116407920692102 

R^2:  -125.58357636121319 
MAE 0.6575962945842915
--------------------- 9 8 ---------------------
SNR: 0.8548777664534848 



### Testing regressor trained using noisy and desynchronized data on noisy and desynchronized data

In [5]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_noisy_desynch_regressor.h5"))

for delay in range(1,10):
  for noise_i in range(10):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateRegressor(model, t, levels_test)

--------------------- 1 0 ---------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -3.1282457350786235 
MAE 0.10108429704430033
--------------------- 1 1 ---------------------
SNR: 54.659191679151576 

R^2:  -2.252862649838847 
MAE 0.08702150119351507
--------------------- 1 2 ---------------------
SNR: 13.67285952979121 

R^2:  -1.2025113535887377 
MAE 0.0695919176470768
--------------------- 1 3 ---------------------
SNR: 6.07748155651528 

R^2:  -0.4653353493121166 
MAE 0.05714825800246973
--------------------- 1 4 ---------------------
SNR: 3.4170100733206645 

R^2:  -0.06230222386883533 
MAE 0.050420436919255296
--------------------- 1 5 ---------------------
SNR: 2.18697764822698 

R^2:  -0.05268219930283902 
MAE 0.05007824608715161
--------------------- 1 6 ---------------------
SNR: 1.5199815724362824 

R^2:  -0.42008253230698966 
MAE 0.05625899576405935
--------------------- 1 7 ---------------------
SNR: 1.115796008857038 

R^2:  -1.2176570633297308 
MAE 0.06951409403576816
--------------------- 1 8 ---------------------
SNR: 0.854474126034

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -3.2030031487636323 
MAE 0.10229504390287697
--------------------- 2 1 ---------------------
SNR: 54.66736025776541 

R^2:  -2.3286381226706836 
MAE 0.08828095271902638
--------------------- 2 2 ---------------------
SNR: 13.672364737627111 

R^2:  -1.2396014734401288 
MAE 0.07017927930304667
--------------------- 2 3 ---------------------
SNR: 6.078403076971928 

R^2:  -0.484030059569194 
MAE 0.057404428826669074
--------------------- 2 4 ---------------------
SNR: 3.4192364651185385 

R^2:  -0.0750147349943564 
MAE 0.05040647641069252
--------------------- 2 5 ---------------------
SNR: 2.1883058306605534 

R^2:  -0.04891526014537528 
MAE 0.04990328376806486
--------------------- 2 6 ---------------------
SNR: 1.5195305213647994 

R^2:  -0.4219498254291252 
MAE 0.056443787114953854
--------------------- 2 7 ---------------------
SNR: 1.1161943040240419 

R^2:  -1.2011506364480184 
MAE 0.06935681563930779
--------------------- 2 8 ---------------------
SNR: 0.85418899

...

SNR: inf 

R^2:  -3.440579351654441 
MAE 0.10606129783859912
--------------------- 7 1 ---------------------
SNR: 54.714115425767815 

R^2:  -2.4438164640062516 
MAE 0.09018045143454236
--------------------- 7 2 ---------------------
SNR: 13.673179983501841 

R^2:  -1.3328888995161345 
MAE 0.07177805755652561
--------------------- 7 3 ---------------------
SNR: 6.076927065975525 

R^2:  -0.5064584148939439 
MAE 0.05768981219235125
--------------------- 7 4 ---------------------
SNR: 3.4166709686492682 

R^2:  -0.08098171065211934 
MAE 0.05035971883589513
--------------------- 7 5 ---------------------
SNR: 2.1871387429483597 

R^2:  -0.0473706890669392 
MAE 0.04989315232473761
--------------------- 7 6 ---------------------
SNR: 1.5192400639302766 

R^2:  -0.44265623975426904 
MAE 0.05642701940070354
--------------------- 7 7 ---------------------
SNR: 1.1164423448142164 

R^2:  -1.2634430105888224 
MAE 0.07048904353897564
--------------------- 7 8 ---------------------
SNR: 0.85470356

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -3.4656562617011604 
MAE 0.10633711625683263
--------------------- 8 1 ---------------------
SNR: 54.74710174446798 

R^2:  -2.4815551074583135 
MAE 0.09077743588550458
--------------------- 8 2 ---------------------
SNR: 13.685172128596243 

R^2:  -1.3043725034952782 
MAE 0.07132332778641842
--------------------- 8 3 ---------------------
SNR: 6.076833961271539 

R^2:  -0.5011454532950079 
MAE 0.05801766479181742
--------------------- 8 4 ---------------------
SNR: 3.417665009965912 

R^2:  -0.08529110526094907 
MAE 0.050627942818874117
--------------------- 8 5 ---------------------
SNR: 2.1897800121027067 

R^2:  -0.05520115952079019 
MAE 0.05019748028435578
--------------------- 8 6 ---------------------
SNR: 1.5189316226238827 

R^2:  -0.5096849923928204 
MAE 0.05783325877463329
--------------------- 8 7 ---------------------
SNR: 1.1159703627967548 

R^2:  -1.3262311684145502 
MAE 0.07144719532884228
--------------------- 8 8 ---------------------
SNR: 0.85430348

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -3.439638886023725 
MAE 0.10583395086776899
--------------------- 9 1 ---------------------
SNR: 54.70693142870829 

R^2:  -2.4506740916137146 
MAE 0.09035882720624064
--------------------- 9 2 ---------------------
SNR: 13.671055951912964 

R^2:  -1.3143000357227477 
MAE 0.07145907319645552
--------------------- 9 3 ---------------------
SNR: 6.078087158216372 

R^2:  -0.5019644103682785 
MAE 0.057881279009854354
--------------------- 9 4 ---------------------
SNR: 3.418861820544086 

R^2:  -0.08776088545461813 
MAE 0.05075456049347497
--------------------- 9 5 ---------------------
SNR: 2.1881199055504714 

R^2:  -0.07544156630849219 
MAE 0.05046947201757174
--------------------- 9 6 ---------------------
SNR: 1.5190217583973444 

R^2:  -0.5063528592500848 
MAE 0.057523670152994484
--------------------- 9 7 ---------------------
SNR: 1.1159108390354227 

R^2:  -1.3899082403768834 
MAE 0.0727468357728666
--------------------- 9 8 ---------------------
SNR: 0.853661505